In [1]:
import torch
import numpy as np
import nmmo
from nmmo import config
from neurips2022nmmo import CompetitionConfig, TeamBasedEnv
from nmmo.io import action

class TrainConfig(CompetitionConfig):
    MAP_N = 2

env = TeamBasedEnv(TrainConfig())

obs = env.reset()

obs,reward,done,info = env.step({})

100%|██████████| 2/2 [00:00<00:00, 17.38it/s]

Generating 2 maps


In [2]:
import yaml
from easydict import EasyDict

with open("./scripts/config/config.yml", 'r') as f:
    arg_file = yaml.full_load(f)
args = EasyDict(arg_file)

In [3]:
for _ in range(5):
    obs,reward,done,info = env.step({})

from utils.envParser import ObservationParser

In [4]:
i=7
myteam=args.self_group_id

parser = ObservationParser()
entities, available, order_in_obs, group_id, global_id, cnt, tiles  = parser.parse(observations=obs[myteam], args=args)
entities

array([[0.03808594, 0.015625  , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.03222656, 0.015625  , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.03417969, 0.015625  , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [5]:
cnt

9

In [6]:
group_id[:cnt].reshape(1,-1)

array([[15, 15, 15, 15, 15, 15, 15, 15, 16]])

In [7]:
global_id[:cnt].reshape(1,-1)

array([[120, 123, 122, 121, 124, 125, 126, 127, -79]])

In [8]:
available[:cnt,i].reshape(1,-1)

array([[0, 0, 0, 0, 1, 1, 1, 1, 1]])

In [9]:
order_in_obs[:cnt,i].reshape(1,-1)

array([[0, 0, 0, 0, 4, 3, 2, 0, 1]])

In [10]:
obs[0][i]['Entity']['Continuous'][:int(obs[myteam][i]['Entity']['N'])]

array([[  1.,   8.,   0.,   1.,   0.,   0.,   0.,  16.,  38.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  1.,   5.,   0.,   1.,   0.,   0.,   0.,  16.,  32.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  1.,   6.,   0.,   1.,   0.,   0.,   0.,  16.,  34.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  1.,   7.,   0.,   1.,   0.,   0.,   0.,  16.,  36.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.]], dtype=float32)

In [11]:
from model.EntityEncoder import EntityEncoder
e_encoder = EntityEncoder(16)

In [12]:
from model.SpatialEncoder import SpatialEncoder
s_encoder = SpatialEncoder(args = args)

In [22]:
s = torch.Tensor(entities).unsqueeze(dim=0)
n = torch.tensor([cnt])
t = [torch.Tensor(tiles)]
g = torch.tensor(group_id).unsqueeze(dim=0)

sp_embed = s_encoder.forward(t, g)
sp_embed[0,:cnt].sum(axis=-1)

tensor([141.9759, 141.9759, 141.9759, 141.9759, 147.1165, 140.7313, 139.4522,
        155.7134,   0.0000], grad_fn=<SumBackward1>)

In [21]:
sp_embed.shape

torch.Size([1, 512, 256])

In [29]:
en_embed = e_encoder.forward(s,g,sp_embed,n)
en_embed[0,:cnt].sum(-1)

tensor([46.1565, 46.1579, 46.1574, 46.1570, 41.6435, 42.9042, 46.9774, 41.7789,
        44.2804], grad_fn=<SumBackward1>)

In [23]:
en_embed.shape

torch.Size([1, 512, 256])

In [15]:
actions_by_team = {
    0: {
        0:  {
            action.Move: {
                action.Direction: 0
            },
            # action.Attack:{
            #     action.Style: 0,
            #     action.Target: 1
            # },
        }
    }
}
# obs,reward,done,info = env.step(actions_by_team)
obs = env.reset()

print(obs)

print(obs[0]['stat'])
print(obs[0].keys())
print(obs[0][0]['Entity']['Continuous'][:int(obs[0][0]['Entity']['N'])])
# print(obs[0][1]['Entity']['Continuous'][:int(obs[0][1]['Entity']['N'])])

print([obs[0][0]['Tile']['Continuous'][i][1] for i in [97,111,112,113,126] ])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [191]:
import numpy as np
import torch

a = np.array([[[0,1],[1,2]],[[2,3],[6,8]]])
a

array([[[0, 1],
        [1, 2]],

       [[2, 3],
        [6, 8]]])

In [192]:
a = torch.Tensor(a)
msk = torch.tensor([[[0,1],[0,1]],[[1,1],[1,1]]],dtype=bool)
~msk

tensor([[[ True, False],
         [ True, False]],

        [[False, False],
         [False, False]]])

In [225]:
b = a.masked_fill_(~msk, -1e9)
c = b.softmax(dim=-1)
c.log()

tensor([[[   -inf,  0.0000],
         [   -inf,  0.0000]],

        [[-1.3133, -0.3133],
         [-2.1269, -0.1269]]])

In [224]:
from torch.distributions import Categorical
m = Categorical(c)
m.sample()

tensor([[1, 1],
        [1, 1]])